# Voice Recognition with Deep Learning

In [1]:
# Data manipulation
import pandas as pd 
import numpy as np 

# For preprocessing data
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# Neural Network
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

## Plotting data
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import os
# set current working dir
os.chdir('/home/adriel_martins/Documents/voice_recognition')

## Preparing the data

Inital data is from the Jurgen Arias's notebook preprocessing of data.

In [2]:
# This numpy array represents 5 features extracted from the signal
# and the respective label
features_label = np.load('Data/features_label.npy', allow_pickle=True)

features_label[0]

(array([-366.63562   ,   96.04874   ,  -69.84662   ,   76.36943   ,
         -24.817017  ,   14.41616   ,    4.2905197 ,   -1.8795805 ,
          13.70058   ,    3.6543803 ,    8.991205  ,   -3.1601498 ,
           5.1049666 ,    2.6289887 ,    1.0638338 ,   -3.784181  ,
           1.1059349 ,    1.8543453 ,    2.2196708 ,    3.0452476 ,
           2.9182453 ,    4.099081  ,    0.39878955,    5.7587442 ,
           0.7864756 ,    3.3036218 ,    1.7996112 ,   -0.8484667 ,
           3.3366387 ,    2.934917  ,    2.9008343 ,    0.5252678 ,
           2.3906317 ,    1.9600887 ,    2.8523176 ,    3.2463658 ,
           3.8902597 ,    4.625333  ,    3.8027694 ,    5.2710724 ],
       dtype=float32),
 array([0.55214745, 0.5712805 , 0.598977  , 0.60294086, 0.61977226,
        0.6546082 , 0.64055675, 0.62010676, 0.586103  , 0.58701015,
        0.6026565 , 0.5885016 ], dtype=float32),
 array([7.36393154e-01, 2.26960018e-01, 1.12153115e-02, 6.72356132e-03,
        1.60283357e-01, 1.04996264e+00,

In [ ]:
def format_feats(in_feats):
    x = in_feats.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    return pd.DataFrame(x_scaled, columns=in_feats.columns)

# Apply some data formatting
def format_data(data):
    # One-hot encode 'Embarked' column
    data = pd.get_dummies(data, columns=['Sex','Embarked'])
    # Drop columns that require additional processing
    data = data.drop(['Name','Ticket','Cabin'], axis=1)
    # Fill null values with the mean of the column
    data.fillna(data.mean(), inplace=True)
    if 'Survived' in data.columns:
        data_y = data['Survived']
        data_x = data.drop(['Survived'], axis=1)
        data_x = format_feats(data_x)
        print('found it')
        return data_x, data_y
    else:
        return format_feats(data)

# This should split the data into our features and our labels
feats, labels = format_data(df)
feats.describe()

In [ ]:
# Splitting between validation and training
X_train, X_validation, Y_train, Y_validation = train_test_split(feats, labels, test_size = 0.1)
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size = 0.3)

In [ ]:
X_train = torch.FloatTensor(X_train.to_numpy())
Y_train = torch.LongTensor(Y_train.to_numpy())

X_test = torch.FloatTensor(X_test.to_numpy())
Y_test = torch.LongTensor(Y_test.to_numpy())

X_validation = torch.FloatTensor(X_validation.to_numpy())
Y_validation = torch.LongTensor(Y_validation.to_numpy())

## Neural Network

In [ ]:
class Classifier(nn.Module):
    def __init__(self, input_feat, output_feat):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(input_feat, 250) 
        self.fc2 = nn.Linear(250, output_feat)
    
    def forward(self, x):        
        x = F.relu(self.fc1(x))
        x = F.log_softmax(self.fc2(x))
        return x

model = Classifier(input_feat=X_train.size()[1], output_feat=2)
model

In [ ]:
model = Classifier(input_feat=X_train.size()[1], output_feat=2)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)

# number of epochs to train the model
n_epochs = 5

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to +infinity

train_losses = []
valid_losses = []

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train() # prep model for training
    lower_bound_index = 0
    for upper_bound_index in range(1,11):
    
        index_range = range(lower_bound_index,
                            (upper_bound_index*(len(X_train)//10)))

        lower_bound_index = upper_bound_index

        x = X_train[index_range]
        y = Y_train[index_range]

        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(x) # log_ps
        # calculate the loss
        loss = criterion(output, y)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*X_train.size(0)

    ######################    
    # validate the model #
    ######################
    model.eval() # prep model for evaluation
    lower_bound_index = 0
    for upper_bound_index in range(1,11):
    
        index_range = range(lower_bound_index,
                            (upper_bound_index*(len(X_validation)//10)))

        lower_bound_index = upper_bound_index

        x = X_validation[index_range]
        y = Y_validation[index_range]

        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(x) # log_ps
        # calculate the loss
        loss = criterion(output, y) 
        # update running validation loss 
        valid_loss += loss.item()*X_validation.size(0)
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(X_train)
    train_losses.append(train_loss)
    valid_loss = valid_loss/len(X_validation)
    valid_losses.append(valid_loss)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = valid_loss

In [ ]:
model = Classifier(X_train.size()[1], 2)
model.load_state_dict(torch.load('model.pt'))

In [ ]:
# initialize lists to monitor test loss and accuracy
test_loss = 0
accuracy = 0
test_losses = []

model.eval() # prepare model for evaluation
# To speed up calculations, forget the gradients, etc.
with torch.no_grad():
    lower_bound_index = 0
    for upper_bound_index in range(1,11):
    
        index_range = range(lower_bound_index,
                            (upper_bound_index*(len(X_validation)//10)))

        lower_bound_index = upper_bound_index

        x = X_test[index_range]
        y = Y_test[index_range]
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(x) # log_ps
        # calculate the loss
        loss = criterion(output, y)
        # update test loss 
        test_loss += loss.item()*x.size(0)
        test_losses.append(test_loss)

        ps = torch.exp(output) # cuz our model outputs log-probability
        top_p, top_class = ps.topk(1, dim = 1)

        equals = (top_class == y.view(*top_class.shape))
        accuracy += torch.mean(equals.type(torch.FloatTensor))/(len(range(1,11)))

        print("Test Loss: {:.3f}.. ".format(test_loss/len(X_test)),
              "Test Accuracy: {:.3f}".format(accuracy))

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(valid_losses, label='Validation loss')
plt.legend(frameon=False)